In [69]:
import yaml
def dump_yaml(data, filename):
    Dumper = yaml.SafeDumper
    Dumper.ignore_aliases = lambda self, data: True
    with open(filename, 'w', encoding='utf8') as f:
        yaml.dump(data, f, Dumper = Dumper,
                  default_flow_style=False, allow_unicode=True)


In [107]:
def update_mappings(r):
    to_iso = "../data-raw/afrobarometer_to_iso/r%d.yml" % r
    to_wals = "../data-raw/afrobarometer_to_wals/r%d.yml" % r
    out = "../data-raw/afrobarometer-mappings/r%d.yml" % r
    with open(to_wals, 'r') as f:
        data_wals = yaml.load(f)

    with open(to_iso, 'r') as f:
        data_iso = yaml.load(f)
        
    newdata = {}
    for row in data_iso:
        try:
            key = (row['lang_id'], tuple(sorted(row['question'])))
        except KeyError as e:
            print(row)
            raise e
        row['wals_code'] = None
        newdata[key] = row   
        
    for row in data_wals:
        try:
            key = (row['lang_id'], tuple(sorted(row['question'])))
        except KeyError as e:
            print(row)
            raise e
        try:
            newdata[key]['wals_code'] = row['wals_code']
        except KeyError as e:
            print(row)
            # raise e

    newdata = sorted([x for x in newdata.values()], key = lambda x: (x['lang_id'], tuple(sorted(x['question']))))
    for row in newdata:
        row['question'] = sorted(row['question'])
        row['iso_639_3'] = sorted(row['iso_639_3'])
        if row['wals_code'] is not None:
            row['wals_code'] = sorted(row['wals_code'])
        if 'note' in row and row['note'] is not None:
            row['note'] = row['note'].strip()
    
    dump_yaml(newdata, out)

Add WALS mappings

In [4]:
import yaml
from yaml.constructor import ConstructorError

try:
    from yaml import CLoader as Loader
except ImportError:
    from yaml import Loader


def no_duplicates_constructor(loader, node, deep=False):
    """Check for duplicate keys."""

    mapping = {}
    for key_node, value_node in node.value:
        key = loader.construct_object(key_node, deep=deep)
        value = loader.construct_object(value_node, deep=deep)
        if key in mapping:
            raise ConstructorError("while constructing a mapping", node.start_mark,
                                   "found duplicate key (%s)" % key, key_node.start_mark)
        mapping[key] = value

    return loader.construct_mapping(node, deep)

yaml.add_constructor(yaml.resolver.BaseResolver.DEFAULT_MAPPING_TAG, no_duplicates_constructor)


for r in range(1, 7):
    try:
        with open("../data-raw/afrobarometer-mappings/r%d.yml" % r, 'r') as f:
            yaml.load(f)
    except ConstructorError as e:
        print(r)
        print(e)

6
while constructing a mapping
  in "../data-raw/afrobarometer-mappings/r6.yml", line 760, column 3
found duplicate key (note)
  in "../data-raw/afrobarometer-mappings/r6.yml", line 769, column 3


In [154]:
import json
with open("../data-raw/ethnologue-tree.json", 'r') as f:
    ethnologue = json.load(f)

In [167]:
def better_tree(key, tree):
    newtree = {k: v for k, v in tree.items() if k not in ['subgroups', 'languages']}
    newtree['path'] = key
    newtree['language'] = 'iso_code' in tree
    newtree['children'] = []
    for i in ('languages', 'subgroups'):
        if i in tree and len(tree[i]):
            try:
                for k, v in tree[i].items():
                    newtree['children'] += [better_tree(k, v)]
            except AttributeError as e:
                raise e
    return newtree


In [168]:
ethnologue_tree = [better_tree(k, v) for k, v in ethnologue.items()]

Generate paths for each ethnologue language

In [188]:
def build_paths(node, root = None):
    if node['language']:
        return [{'lang': node['iso_code'], 'path': []}]
    else:
        out = []
        name = node['name']
        for chld in node['children']:
            for x in build_paths(chld):
                x['path'] = [name]
                out.append(x)
        return out
        
lang_paths = {}
for family in [build_paths(x) for x in ethnologue_tree]:
    for x in family:
        lang_paths[x['lang']] = {'path': x['path'], 'family': x['path'][-1]}

Generate data with distances between languages in the ethnologue. This distance is directed. It is the distance to the closest shared ancestor on the tree.

In [211]:
import csv
with open("../data-raw/ethnologue_dists.csv", 'w') as f:
    writer = csv.DictWriter(f, fieldnames=('from', 'to', 'distance'))
    writer.writeheader()
    writer.writerows(lang_dists)

In [1]:
import datapackage

In [2]:
import goodtables